In [1]:
!nvidia-smi

Tue Dec 31 16:11:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [3]:
%cd {HOME}
!git clone https://github.com/IDEA-Research/GroundingDINO.git
%cd {HOME}/GroundingDINO
!pip install -q -e .
!pip install -q roboflow

/content
Cloning into 'GroundingDINO'...
remote: Enumerating objects: 463, done.
remote: Counting objects: 100% (240/240), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 463 (delta 176), reused 137 (delta 137), pack-reused 223 (from 1)
Receiving objects: 100% (463/463), 12.87 MiB | 30.65 MiB/s, done.
Resolving deltas: 100% (241/241), done.
/content/GroundingDINO
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.7 MB/s eta 0:00:00


In [4]:
import os

CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
print(CONFIG_PATH, "; exist:", os.path.isfile(CONFIG_PATH))

/content/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py ; exist: True


In [5]:
%cd {HOME}
!mkdir {HOME}/weights
%cd {HOME}/weights

!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth

/content
/content/weights


In [6]:
import os

WEIGHTS_NAME = "groundingdino_swint_ogc.pth"
WEIGHTS_PATH = os.path.join(HOME, "weights", WEIGHTS_NAME)
print(WEIGHTS_PATH, "; exist:", os.path.isfile(WEIGHTS_PATH))

/content/weights/groundingdino_swint_ogc.pth ; exist: True


In [8]:
%cd {HOME}/GroundingDINO

from groundingdino.util.inference import load_model, load_image, predict, annotate

model_GD = load_model(CONFIG_PATH, WEIGHTS_PATH)

/content/GroundingDINO


final text_encoder_type: bert-base-uncased


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [16]:
import os
import cv2
import supervision as sv

def process_image_and_crop(
    image_path,
    model,
    text_prompt="Shoe Tag",
    box_threshold=0.35,
    text_threshold=0.25,
    output_dir="."
):
    """
    1) Load image from given path.
    2) Use Grounding DINO's predict(...) to get bounding boxes in [cx, cy, w, h] normalized format.
    3) Convert and crop the image by these boxes.
    4) Save the cropped image with suffix '_Cropped' in the same directory or given output_dir.

    :param image_path: path to the input image
    :param model: a loaded Grounding DINO model object
    :param text_prompt: text prompt used by Grounding DINO to get bounding boxes
    :param box_threshold: bounding box threshold
    :param text_threshold: text threshold
    :param output_dir: directory where cropped image(s) will be saved
    """
    # 1) Load the image
    image_source, image = load_image(image_path)  # from your existing code

    # 2) Run the Grounding DINO prediction
    boxes, logits, phrases = predict(
        model=model,
        image=image,
        caption=text_prompt,
        box_threshold=box_threshold,
        text_threshold=text_threshold
    )

    # 3) Perform cropping
    #    boxes are center-based: [cx, cy, w, h] in [0..1]
    #    We'll convert them to pixel coords, then crop.

    # Create output directory if not exists
    os.makedirs(output_dir, exist_ok=True)

    # Derive a base filename, e.g. from "T4.png" -> "T4"
    base_name = os.path.splitext(os.path.basename(image_path))[0]

    # For each box, crop the region and save
    height, width = image_source.shape[:2]
    for i, box in enumerate(boxes):
        cx, cy, w, h = map(float, box)

        # Unnormalize
        cx_abs = cx * width
        cy_abs = cy * height
        w_abs  = w  * width
        h_abs  = h  * height

        # Calculate corners (x1,y1,x2,y2)
        x1 = int(cx_abs - 0.5 * w_abs)
        y1 = int(cy_abs - 0.5 * h_abs)
        x2 = int(cx_abs + 0.5 * w_abs)
        y2 = int(cy_abs + 0.5 * h_abs)

        # Clamp to boundaries
        x1 = max(0, min(x1, width - 1))
        x2 = max(0, min(x2, width))
        y1 = max(0, min(y1, height - 1))
        y2 = max(0, min(y2, height))

        # Check validity
        if x2 <= x1 or y2 <= y1:
            print(f"Skipping invalid box: {box}")
            continue

        # Crop
        cropped_img = image_source[y1:y2, x1:x2]

        if cropped_img.size == 0:
            print(f"Skipping empty crop: {box}")
            continue

        # Build save path: e.g. T4_Cropped_0.png, T4_Cropped_1.png, etc.
        crop_name = f"{base_name}_Cropped_{i}.png"
        crop_path = os.path.join(output_dir, crop_name)
        cv2.imwrite(crop_path, cropped_img)
        print(f"Saved crop to {crop_path}")


In [17]:
# Example usage:

IMAGE_NAME = "T2.png"
IMAGE_PATH = os.path.join(HOME, "data", IMAGE_NAME)
process_image_and_crop(
    image_path=IMAGE_PATH,
    model=model_GD,
    text_prompt="Shoe Tag",
    box_threshold=0.35,
    text_threshold=0.25,
    output_dir="/content/data"
)


Saved crop to /content/data/T2_Cropped_0.png
